In [ ]:
!python /content/drive/MyDrive/causality-guided-Transformer/PAIN_Main_MAVEN_ERE.py \
 -data /content/drive/MyDrive/causality-guided-Transformer/data/MAVEN_ERE/  \
 -prior /content/drive/MyDrive/causality-guided-Transformer/prior/MAVEN_ERE/sparse/ \
 -epoch 1\
 -batch_size 16\
 -d_model 512 \
 -d_inner 256 \
 -d_k 256 \
 -d_v 256 \
 -n_head 4 \
 -n_layers 4 \
 -dropout 0.1 \
 -lr 1e-4 \
 -num_samples 1

In [ ]:
import subprocess
from itertools import product
import pandas as pd

def fine_tune_model(epoch, batch_size, d_model, d_inner, d_k, d_v, n_head, n_layers, dropout, lr, num_samples, total_epochs):
    print(f"Running fine-tuning with parameters: "
          f"epoch={epoch}, batch_size={batch_size}, d_model={d_model}, "
          f"d_inner={d_inner}, d_k={d_k}, d_v={d_v}, n_head={n_head}, n_layers={n_layers}, "
          f"dropout={dropout}, lr={lr}, num_samples={num_samples}")

    train_loglikelihood = None
    val_loglikelihood = None
    test_loglikelihood = None

    def run_command(command):
        print(f"Running command: {' '.join(command)}")
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        output = result.stdout
        print(result.stdout)
        return output

    for i in range(total_epochs):
        print(f"[Epoch {i+1}/{total_epochs}] Starting fine-tuning...")

        command = [
            'python', '/content/drive/MyDrive/causality-guided-Transformer/PAIN_Main_MAVEN_ERE.py',
            '-data', '/content/drive/MyDrive/causality-guided-Transformer/data/MAVEN_ERE/',
            '-prior', '/content/drive/MyDrive/causality-guided-Transformer/prior/MAVEN_ERE/sparse/',
            '-epoch', str(epoch),
            '-batch_size', str(batch_size),
            '-d_model', str(d_model),
            '-d_inner', str(d_inner),
            '-d_k', str(d_k),
            '-d_v', str(d_v),
            '-n_head', str(n_head),
            '-n_layers', str(n_layers),
            '-dropout', str(dropout),
            '-lr', str(lr),
            '-num_samples', str(num_samples)
        ]

        output = run_command(command)

        for line in output.splitlines():
            if '(Training)' in line:
                train_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
                # print(f"[ Epoch {i+1} ] - (Training) loglikelihood: {train_loglikelihood:.4f}")

            elif '(Validation)' in line:
                val_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
                # print(f"[ Epoch {i+1} ] - (Validation) loglikelihood: {val_loglikelihood:.4f}")

            elif '(Test)' in line:
                test_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
                # print(f"[ Epoch {i+1} ] - (Test) loglikelihood: {test_loglikelihood:.4f}")

        print(f"[ Epoch {i+1} ] - [Info] Maximum validation loglikelihood: {val_loglikelihood:.4f}")
        print(f"Test log likelihood is {test_loglikelihood:.4f}")

    return {
        'epoch': epoch,
        'batch_size': batch_size,
        'd_model': d_model,
        'd_inner': d_inner,
        'd_k': d_k,
        'd_v': d_v,
        'n_head': n_head,
        'n_layers': n_layers,
        'dropout': dropout,
        'lr': lr,
        'num_samples': num_samples,
        'train_loglikelihood': train_loglikelihood,
        'val_loglikelihood': val_loglikelihood,
        'test_loglikelihood': test_loglikelihood
    }

epochs = [1, 3, 5]
batch_sizes = [32, 64, 128]
d_models = [256, 512, 1024]
d_inners = [128, 256, 512]
d_ks = [128, 256]
d_vs = [128, 256]
n_heads = [2, 4, 8]
n_layers = [2, 4, 6]
dropouts = [0.1, 0.2, 0.3]
lrs = [1e-4, 1e-5, 1e-3]
num_samples_list = [1, 3, 5]


hyperparameter_combinations = list(product(epochs,
                                           batch_sizes,
                                           d_models,
                                           d_inners,
                                           d_ks,
                                           d_vs,
                                           n_heads,
                                           n_layers,
                                           dropouts,
                                           lrs,
                                           num_samples_list))

results = []
for params in hyperparameter_combinations:
    print(f"Now running model with hyperparameters: {params}")
    result = fine_tune_model(*params, total_epochs=params[0])
    if result:
        results.append(result)

df = pd.DataFrame(results)
df.to_csv('PAIN_fine_tuning_loglikelihood_results.csv', index=False)
